In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.chdir(r'C:\Users\82104\DM\project')

from tqdm import notebook
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.distance import cdist, pdist

import warnings
warnings.filterwarnings('ignore')

#### (1차)구별로 했을 때 들어가야하는 변수 => 논문 의거x & 구별 데이터 => Clustering


- 운행효율성, 물동량 수, 타켓인구, 매출액, 점포 수, 공시지가

#### (2차) 입지 후보군(구별)에서 최적입지 선정할 때 사용되는 변수 => 논문 의거o & 주유소별 데이터 => Classification


- 개별공시지가, 공급처 접근성(east, west, koreaDC), 면적, 근처 지역 인구수, 생산가능인구(인력출원여부)

We will use some datasets, subtract required column and merge by distrcit.(Gangnam, Ganbuk, Gangdong, ...)  
First, Find the distrcit where MFC(Micro Fullfillment Center) is needed with clustering.  
Second, Set up the optimal center in district by using Score, and then make classification model(Self-supervising)

In [2]:
# There are datasets for using first step.
df_pop = pd.read_csv(r'주민등록인구(연령별_동별)_20221001180232.csv', encoding ='UTF8') # subtract popAll, pop2040 (population)
df_logi = pd.read_csv(r'자치구단위 월별 착지 데이터 (1).csv', encoding ='UTF8') # subtract logistics(traffic volume)
df_sales = pd.read_csv('서울시_상권_매출액.csv', encoding='euc-kr') # subtract sales & num_store
df_citycode = pd.read_csv('서울시 건축물대장 법정동 코드정보.csv', encoding='euc-kr')
df_salesarea = pd.read_csv('서울시 우리마을가게 상권분석서비스(상권영역).csv', encoding='euc-kr')
df_cjdata = pd.read_csv('CJ택배 운행량 월별 통계.csv', encoding='euc-kr') # subtract delivery distance, the number of delivery, delivery time, the number of waypoint.
df_estate = pd.read_csv(r'국토교통부_표준지공시지가_20220527.csv', encoding ='UTF8') # land value by public announcement.

In [3]:
df_pop.head(3)

,동별,항목,소계,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,30~34세,...,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세 이상
0,종로구,계,153255,2749,4283,5013,5621,12746,14661,11200,...,12292,11791,9046,6369,5348,4282,2130,749,206,40.0
1,종로구,한국인,143624,2609,4094,4819,5357,9759,12915,10397,...,11816,11492,8913,6304,5333,4268,2122,745,206,40.0
2,종로구,외국인,9631,140,189,194,264,2987,1746,803,...,476,299,133,65,15,14,8,4,0,0.0


In [4]:
df_pop.isnull().sum()

동별         0
항목         0
소계         0
0~4세       0
5~9세       0
10~14세     0
15~19세     0
20~24세     0
25~29세     0
30~34세     0
35~39세     0
40~44세     0
45~49세     0
50~54세     0
55~59세     0
60~64세     0
65~69세     0
70~74세     0
75~79세     0
80~84세     0
85~89세     0
90~94세     0
95~99세     0
100세 이상    1
dtype: int64

In [5]:
df_pop[df_pop['100세 이상'].isnull()]

,동별,항목,소계,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,30~34세,...,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세 이상
74,강동구,외국인,3866,78,96,76,52,204,463,517,...,438,251,84,38,14,3,3,1,0,NaN


There is a null value in Gangdong-Gu. but since rare people are over 100 years old in the world, it counted with zero.

In [6]:
df_pop.loc[74, '100세 이상'] = 0

In [7]:
df_pop['100세 이상'] = df_pop['100세 이상'].astype(int)

In [8]:
df_pop[(df_pop.동별 == '강동구') & (df_pop.항목 == '외국인')]

,동별,항목,소계,0~4세,5~9세,10~14세,15~19세,20~24세,25~29세,30~34세,...,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세 이상
74,강동구,외국인,3866,78,96,76,52,204,463,517,...,438,251,84,38,14,3,3,1,0,0


In [9]:
# subtract population 20~40 
pop2040 = df_pop[df_pop.항목 =='계'][['동별', '항목', '20~24세', '25~29세', '30~34세', '35~39세', '40~44세', '45~49세']].set_index('동별').sum(axis=1)
pop2040

동별
종로구      69930
중구       61688
용산구     112398
성동구     136671
광진구     175113
동대문구    166805
중랑구     170092
성북구     199990
강북구     122596
도봉구     126118
노원구     215172
은평구     207289
서대문구    147577
마포구     191766
양천구     188467
강서구     275023
구로구     184583
금천구     111478
영등포구    192580
동작구     188778
관악구     269128
서초구     185381
강남구     247056
송파구     307780
강동구     206525
dtype: int64

In [10]:
# logistic dataset.
df_logi.head(3)

,기준년월(BASE_DY),자치구(ADDR),대분류_착지물동량_가구/인테리어(E_C_01),대분류_착지물동량_기타(E_C_02),대분류_착지물동량_도서/음반(E_C_03),대분류_착지물동량_디지털/가전(E_C_04),대분류_착지물동량_생활/건강(E_C_05),대분류_착지물동량_스포츠/레저(E_C_06),대분류_착지물동량_식품(E_C_07),대분류_착지물동량_출산/육아(E_C_08),...,중분류_착지물동량_바디케어(E_C_11_04),중분류_착지물동량_베이스메이크업(E_C_11_05),중분류_착지물동량_뷰티소품(E_C_11_06),중분류_착지물동량_색조메이크업(E_C_11_07),중분류_착지물동량_선케어(E_C_11_08),중분류_착지물동량_스킨케어(E_C_11_09),중분류_착지물동량_클렌징(E_C_11_10),중분류_착지물동량_향수(E_C_11_11),중분류_착지물동량_헤어스타일링(E_C_11_12),중분류_착지물동량_헤어케어(E_C_11_13)
0,202103,강남구,25035,281991,27333,51770,198635,13054,173813,14363,...,12870,5184,5819,15549,6283,25652,9804,2174,3588,5058
1,202104,서초구,19508,259466,48374,38415,165677,17343,175048,12809,...,14142,2216,7462,3222,5000,16368,8505,1938,3307,12029
2,202105,서대문구,16904,384154,29689,46772,76597,17717,466881,14987,...,10622,4462,3432,6019,6032,18570,5389,1509,2114,11127


In [11]:
df_logi.isnull().sum()

기준년월(BASE_DY)                  0
자치구(ADDR)                      0
대분류_착지물동량_가구/인테리어(E_C_01)      0
대분류_착지물동량_기타(E_C_02)           0
대분류_착지물동량_도서/음반(E_C_03)        0
                              ..
중분류_착지물동량_스킨케어(E_C_11_09)      0
중분류_착지물동량_클렌징(E_C_11_10)       0
중분류_착지물동량_향수(E_C_11_11)        0
중분류_착지물동량_헤어스타일링(E_C_11_12)    0
중분류_착지물동량_헤어케어(E_C_11_13)      0
Length: 142, dtype: int64

In [12]:
# sum all logistic traffic 
df_logi['Total_logis'] = df_logi.iloc[:,2:].sum(axis=1)

In [13]:
df_logistic = df_logi[['자치구(ADDR)', 'Total_logis']]
df_logistic.rename(columns={'자치구(ADDR)' : 'District'}, inplace=True)

In [14]:
df_logistic

,District,Total_logis
0,강남구,2439367
1,서초구,2165093
2,서대문구,2575393
3,강서구,2289581
4,성동구,2407838
5,성북구,2484878
6,강북구,2202978
7,구로구,2442377
8,마포구,2067762
9,중구,2397072


In [15]:
df_sales.head(3)

,기준_년_코드,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,주중_매출_비율,주말_매출_비율,주중_매출_금액,주말_매출_금액,남성_매출_금액,여성_매출_금액,주중_매출_건수,주말_매출_건수,점포수
0,2021,2110001,이북5도청사,CS100001,한식음식점,626851391.0,25502,59,41,367617856.0,259233535.0,369379103.0,168952823.0,16355,9147,28
1,2021,2110001,이북5도청사,CS100008,분식전문점,122957138.0,12039,50,50,61070094.0,61887044.0,64010667.0,47678846.0,6435,5604,9
2,2021,2110001,이북5도청사,CS100009,호프-간이주점,5093349.0,177,62,38,3135732.0,1957617.0,3428046.0,1186972.0,106,71,3


In [16]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140830 entries, 0 to 140829
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   기준_년_코드      140830 non-null  int64  
 1   상권_코드        140830 non-null  int64  
 2   상권_코드_명      140830 non-null  object 
 3   서비스_업종_코드    140830 non-null  object 
 4   서비스_업종_코드_명  140830 non-null  object 
 5   분기당_매출_금액    140830 non-null  float64
 6   분기당_매출_건수    140830 non-null  int64  
 7   주중_매출_비율     140830 non-null  int64  
 8   주말_매출_비율     140830 non-null  int64  
 9   주중_매출_금액     140830 non-null  float64
 10  주말_매출_금액     140830 non-null  float64
 11  남성_매출_금액     140830 non-null  float64
 12  여성_매출_금액     140830 non-null  float64
 13  주중_매출_건수     140830 non-null  int64  
 14  주말_매출_건수     140830 non-null  int64  
 15  점포수          140830 non-null  int64  
dtypes: float64(5), int64(8), object(3)
memory usage: 17.2+ MB


In [17]:
df_sales2 = df_sales[['상권_코드', '분기당_매출_금액', '점포수']].groupby('상권_코드').sum().reset_index()
df_sales2

,상권_코드,분기당_매출_금액,점포수
0,1001491,3.444003e+11,6069
1,1001492,1.351039e+12,30461
2,1001493,8.348820e+11,47638
3,1001494,9.341597e+11,19614
4,1001495,6.100271e+11,7811
...,...,...,...
1646,2130322,1.766064e+10,893
1647,2130323,5.402078e+10,1385
1648,2130324,2.274171e+10,871
1649,2130325,2.296665e+10,555


In [18]:
df_salesarea2 = df_salesarea[['상권_코드', '시군구_코드']]
df_salesarea2

,상권_코드,시군구_코드
0,2110753,11545
1,2110427,11350
2,2110755,11545
3,2110739,11545
4,2111017,11710
...,...,...
1666,1001493,11140
1667,1001494,11110
1668,1001495,11710
1669,1001496,11680


In [19]:
df_salesmerge = pd.merge(df_sales2, df_salesarea2)
df_salesmerge

,상권_코드,분기당_매출_금액,점포수,시군구_코드
0,1001491,3.444003e+11,6069,11170
1,1001492,1.351039e+12,30461,11140
2,1001493,8.348820e+11,47638,11140
3,1001494,9.341597e+11,19614,11110
4,1001495,6.100271e+11,7811,11710
...,...,...,...,...
1646,2130322,1.766064e+10,893,11740
1647,2130323,5.402078e+10,1385,11740
1648,2130324,2.274171e+10,871,11740
1649,2130325,2.296665e+10,555,11740


In [20]:
df_citycode2 = df_citycode[['시군구코드', '시군구명']].drop_duplicates()
df_citycode2

,시군구코드,시군구명
0,11740,강동구
2,11290,성북구
3,11530,구로구
4,11305,강북구
10,11410,서대문구
11,11500,강서구
15,11440,마포구
16,11710,송파구
20,11230,동대문구
22,11140,중구


In [21]:
df_citysalesmerge = pd.merge(df_salesmerge, df_citycode2, left_on='시군구_코드', right_on='시군구코드')
df_citysalesmerge

,상권_코드,분기당_매출_금액,점포수,시군구_코드,시군구코드,시군구명
0,1001491,3.444003e+11,6069,11170,11170,용산구
1,2110058,3.433195e+10,922,11170,11170,용산구
2,2110059,8.823601e+09,424,11170,11170,용산구
3,2110060,1.870217e+10,636,11170,11170,용산구
4,2110061,7.722694e+09,233,11170,11170,용산구
...,...,...,...,...,...,...
1646,2130322,1.766064e+10,893,11740,11740,강동구
1647,2130323,5.402078e+10,1385,11740,11740,강동구
1648,2130324,2.274171e+10,871,11740,11740,강동구
1649,2130325,2.296665e+10,555,11740,11740,강동구


In [22]:
df_sales_store = df_citysalesmerge.groupby('시군구명')[['분기당_매출_금액', '점포수']].sum()
df_sales_store

,분기당_매출_금액,점포수
시군구명,,
강남구,1.186065e+13,186288
강동구,2.749058e+12,78406
강북구,1.482913e+12,48726
강서구,2.842142e+12,76835
관악구,2.168390e+12,70236
광진구,2.337916e+12,69547
구로구,2.806535e+12,69651
금천구,3.490343e+12,55524
노원구,1.834499e+12,57391


In [23]:
df_sales_store.reset_index(inplace=True)

In [24]:
df_sales_store

,시군구명,분기당_매출_금액,점포수
0,강남구,1.186065e+13,186288
1,강동구,2.749058e+12,78406
2,강북구,1.482913e+12,48726
3,강서구,2.842142e+12,76835
4,관악구,2.168390e+12,70236
5,광진구,2.337916e+12,69547
6,구로구,2.806535e+12,69651
7,금천구,3.490343e+12,55524
8,노원구,1.834499e+12,57391
9,도봉구,1.210447e+12,40786


In [25]:
df_cjdata.head(3)

,운행년월(DRIVEN_YM),주소(구)(ADDR),운행거리(평균)(DRIVEN_AVR),운행거리(총거리)(DRIVEN_SUM),운행대수(DRIVEN_CNT),운행_총시간(DRIVEN_TIME),경유지_총건수(VIA_CNT),데이터_생성_일시(ETL_INS_DT),데이터_업데이트_일시(ETL_UPD_DT)
0,202011,동대문,10.706256,52579.968458,7122,109180824,402530,2021-07-26 08:45:36.0000000,2021-07-26 08:45:36.0000000
1,202006,용산구,12.758485,85058.221296,3671,66316795,375971,2021-07-26 08:45:36.0000000,2021-07-26 08:45:36.0000000
2,202007,구로구,12.307220,76255.048379,5280,79544164,299260,2021-07-26 08:45:36.0000000,2021-07-26 08:45:36.0000000


In [26]:
df_cjdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   운행년월(DRIVEN_YM)          25 non-null     int64  
 1   주소(구)(ADDR)              25 non-null     object 
 2   운행거리(평균)(DRIVEN_AVR)     25 non-null     float64
 3   운행거리(총거리)(DRIVEN_SUM)    25 non-null     float64
 4   운행대수(DRIVEN_CNT)         25 non-null     int64  
 5   운행_총시간(DRIVEN_TIME)      25 non-null     int64  
 6   경유지_총건수(VIA_CNT)         25 non-null     int64  
 7   데이터_생성_일시(ETL_INS_DT)    25 non-null     object 
 8   데이터_업데이트_일시(ETL_UPD_DT)  25 non-null     object 
dtypes: float64(2), int64(4), object(3)
memory usage: 1.9+ KB


In [27]:
df_cjdata.isnull().sum()

운행년월(DRIVEN_YM)            0
주소(구)(ADDR)                0
운행거리(평균)(DRIVEN_AVR)       0
운행거리(총거리)(DRIVEN_SUM)      0
운행대수(DRIVEN_CNT)           0
운행_총시간(DRIVEN_TIME)        0
경유지_총건수(VIA_CNT)           0
데이터_생성_일시(ETL_INS_DT)      0
데이터_업데이트_일시(ETL_UPD_DT)    0
dtype: int64

In [28]:
df_cjdata.columns = ['DRIVEN_YM', 'ADDR', 'DRIVEN_AVR', 'DRIVEN_SUM', 'DRIVEN_CNT', 'DRIVEN_TIME', 'VIA_CNT', 'ETL_INS_DT', 'ETL_UPD_DT']

In [29]:
df_cjdata.head(3)

,DRIVEN_YM,ADDR,DRIVEN_AVR,DRIVEN_SUM,DRIVEN_CNT,DRIVEN_TIME,VIA_CNT,ETL_INS_DT,ETL_UPD_DT
0,202011,동대문,10.706256,52579.968458,7122,109180824,402530,2021-07-26 08:45:36.0000000,2021-07-26 08:45:36.0000000
1,202006,용산구,12.758485,85058.221296,3671,66316795,375971,2021-07-26 08:45:36.0000000,2021-07-26 08:45:36.0000000
2,202007,구로구,12.307220,76255.048379,5280,79544164,299260,2021-07-26 08:45:36.0000000,2021-07-26 08:45:36.0000000


In [30]:
df_subcjdata = df_cjdata.drop(['DRIVEN_YM', 'ETL_INS_DT', 'ETL_UPD_DT'], axis=1)

In [31]:
df_subcjdata.loc[0,'ADDR'] = '동대문구'
df_subcjdata.loc[10,'ADDR'] = '영등포구'
df_subcjdata.loc[17,'ADDR'] = '서대문구'
df_subcjdata

,ADDR,DRIVEN_AVR,DRIVEN_SUM,DRIVEN_CNT,DRIVEN_TIME,VIA_CNT
0,동대문구,10.706256,52579.968458,7122,109180824,402530
1,용산구,12.758485,85058.221296,3671,66316795,375971
2,구로구,12.307220,76255.048379,5280,79544164,299260
3,노원구,12.789262,54740.653719,5592,142538008,386235
4,서초구,14.762646,96238.630233,6675,49820104,551835
5,마포구,11.978918,63527.487480,3657,59627892,490082
6,성북구,10.397073,80852.665389,6299,101369793,649433
7,강북구,9.560024,50462.112357,4993,150119432,265193
8,도봉구,12.659236,49513.913097,5802,49852948,653489
9,금천구,10.963480,61527.114794,8818,45640312,432666


In [32]:
# 경유지 개수 / 운행대수로 변경 -> n대 차량이 k개 경유지 돌면 k/n 값이 클수록 운행효율에 정비례   
df_subcjdata['driven_eff'] = (df_subcjdata.DRIVEN_SUM / df_subcjdata.DRIVEN_TIME) * (df_subcjdata.VIA_CNT / df_subcjdata.DRIVEN_CNT)

In [33]:
df_subcjdata.drop(['DRIVEN_AVR', 'DRIVEN_SUM', 'DRIVEN_CNT', 'DRIVEN_TIME', 'VIA_CNT'], axis=1, inplace=True)

In [34]:
df_subcjdata

,ADDR,driven_eff
0,동대문구,0.027219
1,용산구,0.131360
2,구로구,0.054334
3,노원구,0.026526
4,서초구,0.159699
5,마포구,0.142776
6,성북구,0.082233
7,강북구,0.017854
8,도봉구,0.111866
9,금천구,0.066146


In [35]:
df_estate.head(3)

,일련번호,시군구,읍면동리,지번구분,본번지,부번지,시도명,시군구명,소재지,지목,...,용도지역2,이용상황,주위환경,지세명,형상명,도로교통,공시지가,지리적위치1,지리적위치2,방위
0,1,11110,10100,1,3,52,서울특별시,종로구,청운동 3-52,공원,...,NaN,공원등,시가지주변 야산지대,급경사지,부정형,세로(불),493500,경기상고,북측 인근,NaN
1,2,11110,10100,1,4,2,서울특별시,종로구,청운동 4-2,임야,...,NaN,자연림,시가지주변 야산지대,완경사지,부정형,맹지,730500,경가상고,북서측 근거리,남향
2,3,11110,10100,1,4,15,서울특별시,종로구,청운동 4-15,대,...,NaN,단독주택,기존 주택지대,완경사지,사다리형,세로(가),4250000,경기상고,북서측 인근,북동향


In [36]:
df_estate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540050 entries, 0 to 540049
Data columns (total 22 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   일련번호    540050 non-null  int64  
 1   시군구     540050 non-null  int64  
 2   읍면동리    540050 non-null  int64  
 3   지번구분    540050 non-null  int64  
 4   본번지     540050 non-null  object 
 5   부번지     540050 non-null  object 
 6   시도명     540050 non-null  object 
 7   시군구명    537528 non-null  object 
 8   소재지     540050 non-null  object 
 9   지목      540050 non-null  object 
 10  면적      540050 non-null  float64
 11  용도지역1   540050 non-null  object 
 12  용도지역2   5398 non-null    object 
 13  이용상황    540050 non-null  object 
 14  주위환경    540050 non-null  object 
 15  지세명     540050 non-null  object 
 16  형상명     540050 non-null  object 
 17  도로교통    540050 non-null  object 
 18  공시지가    540050 non-null  int64  
 19  지리적위치1  540050 non-null  object 
 20  지리적위치2  515324 non-null  object 
 21  방위      20

In [37]:
# 구별 공시지가(단위: 천)
estate = df_estate[df_estate['시도명']=='서울특별시'].groupby('시군구명').공시지가.mean().round(-3).astype(int)
estate

시군구명
강남구     17385000
강동구      5986000
강북구      3705000
강서구      4485000
관악구      5826000
광진구      6014000
구로구      4109000
금천구      3870000
노원구      3911000
도봉구      3451000
동대문구     5198000
동작구      6095000
마포구      9213000
서대문구     5936000
서초구     11141000
성동구      6744000
성북구      4507000
송파구      8497000
양천구      4634000
영등포구     6240000
용산구     10079000
은평구      4391000
종로구      9841000
중구      14812000
중랑구      4089000
Name: 공시지가, dtype: int32

In [38]:
df_table = pd.merge(df_subcjdata, df_sales_store, left_on='ADDR', right_on='시군구명').merge(df_logistic, left_on ='ADDR', right_on='District')

In [39]:
df_table

,ADDR,driven_eff,시군구명,분기당_매출_금액,점포수,District,Total_logis
0,동대문구,0.027219,동대문구,4.447564e+12,66655,동대문구,2400374
1,용산구,0.131360,용산구,6.805869e+12,62840,용산구,2250955
2,구로구,0.054334,구로구,2.806535e+12,69651,구로구,2442377
3,노원구,0.026526,노원구,1.834499e+12,57391,노원구,2507172
4,서초구,0.159699,서초구,6.456707e+12,117682,서초구,2165093
5,마포구,0.142776,마포구,4.087616e+12,99849,마포구,2067762
6,성북구,0.082233,성북구,1.380648e+12,52834,성북구,2484878
7,강북구,0.017854,강북구,1.482913e+12,48726,강북구,2202978
8,도봉구,0.111866,도봉구,1.210447e+12,40786,도봉구,2034893
9,금천구,0.066146,금천구,3.490343e+12,55524,금천구,2201708


In [40]:
df_table['pop2040'] = df_table.ADDR.map(pop2040)
# df_table['popALL'] = df_table.ADDR.map(popALL)
df_table['estate'] = df_table.ADDR.map(estate)
df_table

,ADDR,driven_eff,시군구명,분기당_매출_금액,점포수,District,Total_logis,pop2040,estate
0,동대문구,0.027219,동대문구,4.447564e+12,66655,동대문구,2400374,166805,5198000
1,용산구,0.131360,용산구,6.805869e+12,62840,용산구,2250955,112398,10079000
2,구로구,0.054334,구로구,2.806535e+12,69651,구로구,2442377,184583,4109000
3,노원구,0.026526,노원구,1.834499e+12,57391,노원구,2507172,215172,3911000
4,서초구,0.159699,서초구,6.456707e+12,117682,서초구,2165093,185381,11141000
5,마포구,0.142776,마포구,4.087616e+12,99849,마포구,2067762,191766,9213000
6,성북구,0.082233,성북구,1.380648e+12,52834,성북구,2484878,199990,4507000
7,강북구,0.017854,강북구,1.482913e+12,48726,강북구,2202978,122596,3705000
8,도봉구,0.111866,도봉구,1.210447e+12,40786,도봉구,2034893,126118,3451000
9,금천구,0.066146,금천구,3.490343e+12,55524,금천구,2201708,111478,3870000


In [41]:
df_table.drop(['시군구명', 'ADDR'], axis=1, inplace=True)
df_table

,driven_eff,분기당_매출_금액,점포수,District,Total_logis,pop2040,estate
0,0.027219,4.447564e+12,66655,동대문구,2400374,166805,5198000
1,0.131360,6.805869e+12,62840,용산구,2250955,112398,10079000
2,0.054334,2.806535e+12,69651,구로구,2442377,184583,4109000
3,0.026526,1.834499e+12,57391,노원구,2507172,215172,3911000
4,0.159699,6.456707e+12,117682,서초구,2165093,185381,11141000
5,0.142776,4.087616e+12,99849,마포구,2067762,191766,9213000
6,0.082233,1.380648e+12,52834,성북구,2484878,199990,4507000
7,0.017854,1.482913e+12,48726,강북구,2202978,122596,3705000
8,0.111866,1.210447e+12,40786,도봉구,2034893,126118,3451000
9,0.066146,3.490343e+12,55524,금천구,2201708,111478,3870000


In [42]:
df_table.rename(columns={'분기당_매출_금액' : 'Total_sales', '점포수' : 'Num_store'}, inplace=True)
df_table

,driven_eff,Total_sales,Num_store,District,Total_logis,pop2040,estate
0,0.027219,4.447564e+12,66655,동대문구,2400374,166805,5198000
1,0.131360,6.805869e+12,62840,용산구,2250955,112398,10079000
2,0.054334,2.806535e+12,69651,구로구,2442377,184583,4109000
3,0.026526,1.834499e+12,57391,노원구,2507172,215172,3911000
4,0.159699,6.456707e+12,117682,서초구,2165093,185381,11141000
5,0.142776,4.087616e+12,99849,마포구,2067762,191766,9213000
6,0.082233,1.380648e+12,52834,성북구,2484878,199990,4507000
7,0.017854,1.482913e+12,48726,강북구,2202978,122596,3705000
8,0.111866,1.210447e+12,40786,도봉구,2034893,126118,3451000
9,0.066146,3.490343e+12,55524,금천구,2201708,111478,3870000


In [43]:
df_table = df_table[['District', 'driven_eff', 'Total_logis', 'pop2040', 'Total_sales', 'Num_store', 'estate']]
df_table

,District,driven_eff,Total_logis,pop2040,Total_sales,Num_store,estate
0,동대문구,0.027219,2400374,166805,4.447564e+12,66655,5198000
1,용산구,0.131360,2250955,112398,6.805869e+12,62840,10079000
2,구로구,0.054334,2442377,184583,2.806535e+12,69651,4109000
3,노원구,0.026526,2507172,215172,1.834499e+12,57391,3911000
4,서초구,0.159699,2165093,185381,6.456707e+12,117682,11141000
5,마포구,0.142776,2067762,191766,4.087616e+12,99849,9213000
6,성북구,0.082233,2484878,199990,1.380648e+12,52834,4507000
7,강북구,0.017854,2202978,122596,1.482913e+12,48726,3705000
8,도봉구,0.111866,2034893,126118,1.210447e+12,40786,3451000
9,금천구,0.066146,2201708,111478,3.490343e+12,55524,3870000


In [44]:
df_table.columns = df_table.columns.str.upper()

In [45]:
# Final dataset for using first step.
df_table

,DISTRICT,DRIVEN_EFF,TOTAL_LOGIS,POP2040,TOTAL_SALES,NUM_STORE,ESTATE
0,동대문구,0.027219,2400374,166805,4.447564e+12,66655,5198000
1,용산구,0.131360,2250955,112398,6.805869e+12,62840,10079000
2,구로구,0.054334,2442377,184583,2.806535e+12,69651,4109000
3,노원구,0.026526,2507172,215172,1.834499e+12,57391,3911000
4,서초구,0.159699,2165093,185381,6.456707e+12,117682,11141000
5,마포구,0.142776,2067762,191766,4.087616e+12,99849,9213000
6,성북구,0.082233,2484878,199990,1.380648e+12,52834,4507000
7,강북구,0.017854,2202978,122596,1.482913e+12,48726,3705000
8,도봉구,0.111866,2034893,126118,1.210447e+12,40786,3451000
9,금천구,0.066146,2201708,111478,3.490343e+12,55524,3870000


In [46]:
df_table.head()

,DISTRICT,DRIVEN_EFF,TOTAL_LOGIS,POP2040,TOTAL_SALES,NUM_STORE,ESTATE
0,동대문구,0.027219,2400374,166805,4.447564e+12,66655,5198000
1,용산구,0.131360,2250955,112398,6.805869e+12,62840,10079000
2,구로구,0.054334,2442377,184583,2.806535e+12,69651,4109000
3,노원구,0.026526,2507172,215172,1.834499e+12,57391,3911000
4,서초구,0.159699,2165093,185381,6.456707e+12,117682,11141000


In [47]:
df_table.to_csv(r'data_for_clustering.csv', index=False) # 전처리 완료 파일 export